# Cheese Recommender

## Scraping the data

In [1]:
import pandas as pd 
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
import unicodedata

Recommending cheese based on blah blah blah...  Cheese.com very lenient API

directory https://od-do.agr.gc.ca/canadianCheeseDirectoryDictionaryEn.html#cont  
csv downloaded from https://open.canada.ca/data/dataset/3c16cd48-3ac3-453f-8260-6f745181c83b  
data site discovered via https://toolbox.google.com/datasetsearch/search?query=cheese&docid=J4Hb%2B1hlYvt7KkwYAAAAAA%3D%3D

In [2]:
# function to strip strings of accents, from BartoszKP and oefe on StackOverflow
# https://stackoverflow.com/questions/517923/what-is-the-best-way-to-remove-accents-in-a-python-unicode-string

import unicodedata
def strip_accents(s):
   return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')


In [3]:
#read cheese_posts.csv into 'train'
# data found on https://open.canada.ca/data/dataset/3c16cd48-3ac3-453f-8260-6f745181c83b
# data set discovered via 
can_chz = pd.read_csv('./data/canadianCheeseDirectory.csv')

In [4]:
can_chz.T

,0,1,2,3,4,5,6,7,8,9,...,1441,1442,1443,1444,1445,1446,1447,1448,1449,1450
CheeseId,228,242,301,303,319,350,374,375,376,378,...,2382,2383,2384,2385,2386,2387,2388,2389,2390,2391
CheeseNameEn,NaN,NaN,Provolone Sette Fette (Tre-Stelle),NaN,NaN,Paneer (Northumberland Co-operative),Goat Brie (Woolwich),Goat Cheddar (Woolwich),Goat Mozarella (Woolwich),Goat Feta (Woolwich),...,Brigand,Gavroche,Petit frais aux noix et sirop d'érable,Petits caprice,Petit frais aux canneberges,Knoydart,FRESK-O,Electric Blue,Hip Hop,Super Fresh Cheese Curds
CheeseNameFr,Sieur de Duplessis (Le),Tomme Le Champ Doré,Provolone Sette Fette (Tre-Stelle),Geai Bleu (Le),Gamin (Le),NaN,NaN,NaN,NaN,NaN,...,Brigand,Gavroche,Petit frais aux noix et sirop d'érable,Petits caprice,Petit frais aux canneberges,Knoydart,FRESK-O,Electric Blue,Hip Hop,Super Fresh Cheese Curds
ManufacturerNameEn,NaN,NaN,Tre Stelle (Arla Foods),NaN,NaN,Northumberland Co-operative,Woolwich Dairy,Woolwich Dairy,Woolwich Dairy,Woolwich Dairy,...,NaN,NaN,NaN,NaN,NaN,Knoydart Farm Cheese,Latin Foods Inc.,Blue Harbour Cheese,Blue Harbour Cheese,The Cheese Factory Ltd.
ManufacturerNameFr,Fromages la faim de loup,Fromages la faim de loup,NaN,Fromages la faim de loup,Fromages la faim de loup,NaN,NaN,NaN,NaN,NaN,...,Fromagerie Au Fond Des Bois,Fromagerie Au Fond Des Bois,Fromagerie Au Fond Des Bois,Fromagerie Au Fond Des Bois,Fromagerie Au Fond Des Bois,Knoydart Farm Cheese,Latin Foods Inc.,Blue Harbour Cheese,Blue Harbour Cheese,NaN
ManufacturerProvCode,NB,NB,ON,NB,NB,NB,ON,ON,ON,ON,...,NB,NB,NB,NB,NB,NS,AB,NS,NS,AB
ManufacturingTypeEn,Farmstead,Farmstead,Industrial,Farmstead,Farmstead,Industrial,Industrial,Industrial,Industrial,Industrial,...,Farmstead,Farmstead,Farmstead,Farmstead,Farmstead,Farmstead,Industrial,Artisan,Artisan,Artisan
ManufacturingTypeFr,Fermière,Fermière,Industrielle,Fermière,Fermière,Industrielle,Industrielle,Industrielle,Industrielle,Industrielle,...,Fermière,Fermière,Fermière,Fermière,Fermière,Fermière,Industrielle,Artisanale,Artisanale,Artisanale
WebSiteEn,NaN,NaN,http://www.trestelle.ca/english/,NaN,NaN,http://www.northumberlanddairy.ca/e/0001e.cfm,http://www.woolwichdairy.com,http://www.woolwichdairy.com,http://www.woolwichdairy.com,http://www.woolwichdairy.com,...,http://http://www.aufonddesbois.com/English/in...,http://http://www.aufonddesbois.com/English/in...,http://http://www.aufonddesbois.com/English/in...,http://http://www.aufonddesbois.com/English/in...,http://http://www.aufonddesbois.com/English/in...,http://https://www.knoydartfarm.org,http://www.fresk-o.com,http://www.blueharbourcheese.com,http://www.blueharbourcheese.com,http://www.thecheesefactory.ca
WebSiteFr,NaN,NaN,http://www.trestelle.ca/francais/,NaN,NaN,http://www.northumberlanddairy.ca/f/0001f.cfm,http://www.woolwichdairy.com/french.aspx,http://www.woolwichdairy.com/french.aspx,http://www.woolwichdairy.com/french.aspx,http://www.woolwichdairy.com/french.aspx,...,http://http://www.aufonddesbois.com/Francais/i...,http://http://www.aufonddesbois.com/Francais/i...,http://http://www.aufonddesbois.com/Francais/i...,http://http://www.aufonddesbois.com/Francais/i...,http://http://www.aufonddesbois.com/Francais/i...,NaN,NaN,NaN,NaN,NaN


In [6]:
can_chz['CheeseNameEn'].unique()

array([nan, 'Provolone Sette Fette (Tre-Stelle)',
       'Paneer (Northumberland Co-operative)', 'Goat Brie (Woolwich)',
       'Goat Cheddar (Woolwich)', 'Goat Mozarella (Woolwich)',
       'Goat Feta (Woolwich)', 'Leoni-Grana',
       'Gouda (Jerseyland Organic)', 'Cheddar (Jerseyland Organic)',
       'Grana (Jerseyland Organic)', 'Feta (Jerseyland Organic)',
       'Udderly Organic Goat Gouda (Jerseyland Organic)',
       'Udderly Organic Goat Feta (Jerseyland Organic)', 'Madawaska',
       'Cracked Pepper Verdelait', 'Cumin Verdelait',
       'Garlic and Chive Verdelait', 'Wasabi Verdelait',
       'Comox Camembert', 'Provolone (Silani)', 'Ricotta (Silani)',
       'Gouda (Sylvan Star Ltd.)', 'Niagara Gold', 'Comfort Cream',
       'Cheddar (Balderson)', "Gouda (That Dutchman's Farm)",
       "Dragon's Breath", 'Feta (Black Diamond)', 'Dalhousie',
       'Highland Blue', 'Flower Station', 'Snow Road', 'French Line',
       'Ramembert (Best Baa Dairy)', 'Feta (Best Baa Dairy)',
   

In [7]:
can_chz['CheeseNameFr'].unique()

array(['Sieur de Duplessis (Le)', 'Tomme Le Champ Doré',
       'Provolone Sette Fette (Tre-Stelle)', ..., 'Electric Blue',
       'Hip Hop', 'Super Fresh Cheese Curds'], dtype=object)

In [8]:
can_chz['CheeseNameFr']#.iloc[item[0]]

0                  Sieur de Duplessis (Le)
1                      Tomme Le Champ Doré
2       Provolone Sette Fette (Tre-Stelle)
3                           Geai Bleu (Le)
4                               Gamin (Le)
                       ...                
1446                              Knoydart
1447                               FRESK-O
1448                         Electric Blue
1449                               Hip Hop
1450              Super Fresh Cheese Curds
Name: CheeseNameFr, Length: 1451, dtype: object

In [81]:
can_chz

,CheeseId,CheeseNameEn,CheeseNameFr,ManufacturerNameEn,ManufacturerNameFr,ManufacturerProvCode,ManufacturingTypeEn,ManufacturingTypeFr,WebSiteEn,WebSiteFr,...,Organic,CategoryTypeEn,CategoryTypeFr,MilkTypeEn,MilkTypeFr,MilkTreatmentTypeEn,MilkTreatmentTypeFr,RindTypeEn,RindTypeFr,LastUpdateDate
0,228,NaN,Sieur de Duplessis (Le),NaN,Fromages la faim de loup,NB,Farmstead,Fermière,NaN,NaN,...,0,Firm Cheese,Pâte ferme,Ewe,Brebis,Raw Milk,Lait cru,Washed Rind,Croûte lavée,2016-02-03
1,242,NaN,Tomme Le Champ Doré,NaN,Fromages la faim de loup,NB,Farmstead,Fermière,NaN,NaN,...,0,Semi-soft Cheese,Pâte demi-ferme,Cow,Vache,Raw Milk,Lait cru,Washed Rind,Croûte lavée,2016-02-03
2,301,Provolone Sette Fette (Tre-Stelle),Provolone Sette Fette (Tre-Stelle),Tre Stelle (Arla Foods),NaN,ON,Industrial,Industrielle,http://www.trestelle.ca/english/,http://www.trestelle.ca/francais/,...,0,Firm Cheese,Pâte ferme,Cow,Vache,Pasteurized,Pasteurisé,NaN,NaN,2016-02-03
3,303,NaN,Geai Bleu (Le),NaN,Fromages la faim de loup,NB,Farmstead,Fermière,NaN,NaN,...,0,Veined Cheeses,Pâte persillée,Cow,Vache,Raw Milk,Lait cru,NaN,NaN,2016-02-03
4,319,NaN,Gamin (Le),NaN,Fromages la faim de loup,NB,Farmstead,Fermière,NaN,NaN,...,1,Semi-soft Cheese,Pâte demi-ferme,Cow,Vache,Raw Milk,Lait cru,Washed Rind,Croûte lavée,2016-02-03
5,350,Paneer (Northumberland Co-operative),NaN,Northumberland Co-operative,NaN,NB,Industrial,Industrielle,http://www.northumberlanddairy.ca/e/0001e.cfm,http://www.northumberlanddairy.ca/f/0001f.cfm,...,0,Fresh Cheese,Pâte fraîche,Cow,Vache,Pasteurized,Pasteurisé,NaN,NaN,2016-02-03
6,374,Goat Brie (Woolwich),NaN,Woolwich Dairy,NaN,ON,Industrial,Industrielle,http://www.woolwichdairy.com,http://www.woolwichdairy.com/french.aspx,...,0,Soft Cheese,Pâte molle,Goat,Chèvre,Pasteurized,Pasteurisé,Bloomy Rind,Croûte fleurie,2016-02-03
7,375,Goat Cheddar (Woolwich),NaN,Woolwich Dairy,NaN,ON,Industrial,Industrielle,http://www.woolwichdairy.com,http://www.woolwichdairy.com/french.aspx,...,0,Firm Cheese,Pâte ferme,Goat,Chèvre,Pasteurized,Pasteurisé,NaN,NaN,2016-02-03
8,376,Goat Mozarella (Woolwich),NaN,Woolwich Dairy,NaN,ON,Industrial,Industrielle,http://www.woolwichdairy.com,http://www.woolwichdairy.com/french.aspx,...,0,Semi-soft Cheese,Pâte demi-ferme,Goat,Chèvre,Pasteurized,Pasteurisé,NaN,NaN,2016-02-03
9,378,Goat Feta (Woolwich),NaN,Woolwich Dairy,NaN,ON,Industrial,Industrielle,http://www.woolwichdairy.com,http://www.woolwichdairy.com/french.aspx,...,0,Soft Cheese,Pâte molle,Goat,Chèvre,Pasteurized,Pasteurisé,NaN,NaN,2016-02-03


In [82]:
can_chz['ManufacturingTypeEn'].unique()

array(['Farmstead', 'Industrial', 'Artisan'], dtype=object)

In [83]:
can_chz['ParticularitiesEn'].unique()

array([nan, 'Traditional Indian Cheese, also available : Light Paneer',
       'Rennet free, ripens from the outside in',
       'Medium Aged, rennet free', 'Rennet free',
       'Roasted Red Pepper, Cranberry with Port, Mediterranean, Lemon poppy seed',
       'Available : Plain, Garlic and dill, Mixed herb, Caraway, Cumin, Onion & garlic, Hemp',
       'Available : Plain, onion and garlic, Jalapeno/Habenero, Leicester',
       'Available : Plain or Sundried tomato and oregano and onion',
       'The Royal Agricultural Winter Fair 1st Place',
       'The Royal Agricultural Winter Fair 1st Place 2004',
       'British Empire Cheese Show 2004 1st Place',
       '2006 Canadian Grand Prix Finalist',
       '2006 Canadian Cheese Grand Prix Category Champion "Pasta filata"',
       '2006 Canadian Cheese Grand Prix Finalist',
       '2006, 2009, 2011 Canadian Cheese Grand Prix Category Champion',
       'Awarded second place (silver) at the 2009 American Cheese Society Judging and Competitio

In [84]:
can_chz['MilkTreatmentTypeEn'].unique() # Thermisation is a cooking process less severe then pasteurization

array(['Raw Milk', 'Pasteurized', 'Thermised', nan], dtype=object)

In [85]:
# fill in all of the null values in the english columns with values from the corresponding french features
# then drop the french column, and rename the remaining columns

can_chz['CheeseNameEn'].fillna(can_chz['CheeseNameFr'], inplace=True)
can_chz['ManufacturerNameEn'].fillna(can_chz['ManufacturerNameFr'], inplace=True)
can_chz['ManufacturingTypeEn'].fillna(can_chz['ManufacturingTypeFr'], inplace=True)
can_chz['WebSiteEn'].fillna(can_chz['WebSiteFr'], inplace=True)
can_chz['ParticularitiesEn'].fillna(can_chz['ParticularitiesFr'], inplace=True)
can_chz['FlavourEn'].fillna(can_chz['FlavourFr'], inplace=True)
can_chz['CharacteristicsEn'].fillna(can_chz['CharacteristicsFr'], inplace=True)
can_chz['RipeningEn'].fillna(can_chz['RipeningFr'], inplace=True)
can_chz['CategoryTypeEn'].fillna(can_chz['CategoryTypeFr'], inplace=True)
can_chz['MilkTypeEn'].fillna(can_chz['MilkTypeFr'], inplace=True)
can_chz['MilkTreatmentTypeEn'].fillna(can_chz['MilkTreatmentTypeFr'], inplace=True)
can_chz['RindTypeEn'].fillna(can_chz['RindTypeFr'], inplace=True)


can_chz.drop(columns=['CheeseNameFr', 'ManufacturerNameFr', 'ManufacturingTypeFr', 
                      'WebSiteFr', 'ParticularitiesFr', 'FlavourFr', 'CharacteristicsFr', 'RipeningFr', 
                     'CategoryTypeFr', 'MilkTypeFr', 'MilkTreatmentTypeFr', 'RindTypeFr'], inplace=True)

can_chz.rename(columns={"CheeseNameEn": "Name", "ManufacturerNameEn": "Manufacturer", 
                        "ManufacturingTypeEn": "Source_Type", "WebSiteEn": "Website", 
                        "ParticularitiesEn": "Rando_Details", "FlavourEn": "Flavor", 
                        "CharacteristicsEn": "Characteristics", "RipeningEn": "Age", 
                        "CategoryTypeEn": "Category", "MilkTypeEn": "Animal",
                       "MilkTreatmentTypeEn": "Raw_or_Past", "RindTypeEn": "Rind", 
                       "ManufacturerProvCode": "Provence"}, inplace=True)

In [86]:
# drop columns that provide no value to this project
can_chz.drop(columns=['CheeseId', 'Website', 'LastUpdateDate'], inplace=True)

In [87]:
eda(can_chz)

missing values: 
Name                   0
Manufacturer           0
Provence               0
Source_Type            0
FatContentPercent    409
MoisturePercent      420
Rando_Details        945
Flavor               505
Characteristics      688
Age                  709
Organic                0
Category             144
Animal                21
Raw_or_Past          215
Rind                 594
dtype: int64

dataframe index: 
RangeIndex(start=0, stop=1451, step=1)

dataframe types: 
Name                  object
Manufacturer          object
Provence              object
Source_Type           object
FatContentPercent    float64
MoisturePercent      float64
Rando_Details         object
Flavor                object
Characteristics       object
Age                   object
Organic                int64
Category              object
Animal                object
Raw_or_Past           object
Rind                  object
dtype: object

dataframe shape: 
(1451, 15)

dataframe describe: 
       FatContent

In [88]:
can_chz[can_chz['Animal'].isnull()]

,Name,Manufacturer,Provence,Source_Type,FatContentPercent,MoisturePercent,Rando_Details,Flavor,Characteristics,Age,Organic,Category,Animal,Raw_or_Past,Rind
1204,Wig-Wam,Dépendances (Les),QC,Artisan,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
1228,Mascaret,Fromagerie des Grondines,QC,Artisan,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
1231,Maudite (La),Fromagerie du Champ à la Meule,QC,Farmstead,NaN,NaN,Made with 100% Holstein milk,NaN,NaN,NaN,0,NaN,NaN,Thermised,NaN
1252,Crazy Goat,Fromagerie Les Folies Bergères (La),QC,Artisan,NaN,NaN,NaN,NaN,Greek style Feta,NaN,0,NaN,NaN,NaN,No Rind
1276,Fromage à la Crème (Hilary's Fine Cheeses),Hilary's Fine Cheeses,BC,Artisan,NaN,NaN,NaN,NaN,NaN,NaN,0,Fresh Cheese,NaN,NaN,No Rind
1277,Quark (Hilary's Fine Cheeses),Hilary's Fine Cheeses,BC,Artisan,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
1279,Sacré Bleu,Hilary's Fine Cheeses,BC,Artisan,NaN,NaN,NaN,NaN,NaN,NaN,0,Veined Cheeses,NaN,NaN,NaN
1280,Valley Blue,Hilary's Fine Cheeses,BC,Artisan,NaN,NaN,NaN,NaN,NaN,NaN,0,Veined Cheeses,NaN,NaN,NaN
1281,Santa Lucia Fior' di Latte,International Cheese Company,ON,Industrial,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
1282,Santa Lucia Aged Parmesan (International Chees...,International Cheese Company,ON,Industrial,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN


In [89]:
# set the dataframe to drop any rows where we don't know what animal the milk came from 
#can_chz = can_chz[can_chz['Animal'].notnull()].reset_index(drop=True)
    #### . WAAAAAIT MAYBE FILL WHEN I CAN FIND VIA RESEARCH???

In [90]:
can_chz.isnull().sum()

Name                   0
Manufacturer           0
Provence               0
Source_Type            0
FatContentPercent    409
MoisturePercent      420
Rando_Details        945
Flavor               505
Characteristics      688
Age                  709
Organic                0
Category             144
Animal                21
Raw_or_Past          215
Rind                 594
dtype: int64

In [91]:
can_chz.shape

(1451, 15)

In [92]:
can_chz[can_chz['Name'] == 'Wig-Wam'].isnull().sum() # drop

Name                 0
Manufacturer         0
Provence             0
Source_Type          0
FatContentPercent    1
MoisturePercent      1
Rando_Details        1
Flavor               1
Characteristics      1
Age                  1
Organic              0
Category             1
Animal               1
Raw_or_Past          1
Rind                 1
dtype: int64

In [102]:
can_chz['Name']

0                  Sieur de Duplessis (Le)
1                      Tomme Le Champ Doré
2       Provolone Sette Fette (Tre-Stelle)
3                           Geai Bleu (Le)
4                               Gamin (Le)
                       ...                
1446                              Knoydart
1447                               FRESK-O
1448                         Electric Blue
1449                               Hip Hop
1450              Super Fresh Cheese Curds
Name: Name, Length: 1451, dtype: object

In [104]:
len(can_chz)

1451

In [105]:
can_chz.dropna().shape

(145, 15)